In [1]:
import cv2
import numpy as np
import os
import pytesseract as pt

In [2]:
# image setting
# our model is trained on 640x640 image
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 640

In [59]:
# load image
img = cv2.imread('./test_images/getty_sample.jpg')
cv2.namedWindow('example',cv2.WINDOW_NORMAL) #it allows to resize the window
cv2.imshow('example',img)
cv2.waitKey(0) #allows to display a window for given milliseconds or until any key is pressed
cv2.destroyAllWindows() #if i press any button the window will be closed

In [60]:
#Load Model
net = cv2.dnn.readNetFromONNX('./Model6/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [61]:
#COnvert Image to YOLO format
image = img.copy()
row,col,d = image.shape

#get the max
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image

cv2.namedWindow('example',cv2.WINDOW_NORMAL) #it allows to resize the window
cv2.imshow('example',input_image)
cv2.waitKey(0) #allows to display a window for given milliseconds or until any key is pressed
cv2.destroyAllWindows() #if i press any button the window will be closed

In [62]:
#GET prediction from YOLO model
blob = cv2.dnn.blobFromImage(input_image,1/255,(IMAGE_WIDTH,IMAGE_HEIGHT),swapRB=True,crop=False)
net.setInput(blob)
preds = net.forward()
detections = preds[0]

In [63]:
#FILTER detections based on confidence and probability score
#1st column = center_x , 2nd column = center_y , 3rd col = w , 4th col = h , 5th col = confidence , 6th col = probability
boxes = []
confidences = []

image_w , image_h = input_image.shape[:2]
x_factor = image_w/IMAGE_WIDTH
y_factor = image_h/IMAGE_HEIGHT

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #confidence of detectin licence plate
    if confidence > 0.4:
        class_score = row[5] #probability score of licence plate
        if class_score > 0.25:
            cx , cy , w , h = row[0:4] # get the bounding box dimensions
            
            left = int((cx-0.5*w)*x_factor)
            top = int((cy-0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            box = np.array([left,top,width,height])
            
            confidences.append(confidence)
            boxes.append(box)
        
        #Non-Maximum Suppression
        # its when the bounding box is repeated many time
        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()
        
        index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()
            

In [64]:
#Draw the rectangle box
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf = confidences_np[ind]
    conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
    
    cv2.rectangle(image,(x,y),(x+w,y+w),(255,0,255),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
    
    cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_TRIPLEX,0.7,(255,255,255),1)

cv2.namedWindow('PLATE DETECTION',cv2.WINDOW_NORMAL)
cv2.imshow('PLATE DETECTION',image)
cv2.waitKey(0) #allows to display a window for given milliseconds or until any key is pressed
cv2.destroyAllWindows() #if i press any button the window will be closed